In [43]:
!pip install tensorflow
!pip install keras
!pip install pandas
!pip install nltk
!pip install hazm
!pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Using cached scikit_learn-1.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30.4 MB)
  Using cached scipy-1.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (42.2 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=cc7ee3283f39c6fd13917ec57450919c20883f004cde54e68d54709e357b102f
  Stored in directory: /home/milad/.cache/pip/wheels/9b/13/01/6f3a7fd641f90e1f6

In [26]:
import nltk
import pandas as pd
import numpy as np
from hazm import *
import tensorflow as tf
from keras.models import Sequential, save_model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping

In [27]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/milad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
sentence = "Hello, world!"
print (nltk.word_tokenize(sentence))

['Hello', ',', 'world', '!']


In [29]:
dataset = pd.read_csv("hamshahri.csv")
print(dataset.head())

      DID      Date    Cat                                               Text
0     1S1  75\04\02  adabh  جاودانگي در زندگي گروهي از طريق هنر \nنگاهي به...
1  DID2S1  75\04\02  adabh  رويدادهاي هنري جهان \nنمايشگاه هنر در خدمت ديك...
2  DID3S1  75\04\02  adabh  برديوار نگارخانه ها \nگالري گلستان: \nنمايشگاه...
3  DID4S1  75\04\02  ejtem  بازي را جدي بگيريم \nمطالعه اي مقدماتي پيرامون...
4  DID5S1  75\04\02  elmfa  تخته سياه و غباري كه سترده نمي شود... \nاشاره;...


In [30]:
data_len = dataset.shape[0]
print('Dataset has {} rows.'.format(data_len))

Dataset has 165233 rows.


In [31]:
cats = dataset["Cat"]
text = dataset["Text"]
unique_cats = np.unique(cats)
cat_texts_count = dict.fromkeys(unique_cats, 0)

for cat in cats:
  cat_texts_count[cat] += 1

for cat, cnt in cat_texts_count.items():
  print('Category "{}" contains {} posts.'.format(cat, cnt))

Category ". راسته: . شب كلاه شكلان (Semaeostomae)" contains 1 posts.
Category "Adabh" contains 13 posts.
Category "Aeqts" contains 10 posts.
Category "Akhar" contains 10 posts.
Category "Busiw" contains 9 posts.
Category "Cinew" contains 3 posts.
Category "Cultw" contains 651 posts.
Category "Econo" contains 6 posts.
Category "Eqtes" contains 14 posts.
Category "Globa" contains 4 posts.
Category "Gozar" contains 3 posts.
Category "Kharj" contains 22 posts.
Category "Lifew" contains 3 posts.
Category "Musical" contains 3 posts.
Category "Newsp" contains 9 posts.
Category "Nnaft" contains 6 posts.
Category "Polig" contains 2 posts.
Category "Santj" contains 4 posts.
Category "Sciew" contains 6 posts.
Category "Shari" contains 8 posts.
Category "Siasi" contains 18 posts.
Category "Sporw" contains 50 posts.
Category "Theatre" contains 3 posts.
Category "Thought" contains 2 posts.
Category "abksh" contains 495 posts.
Category "adabh" contains 5171 posts.
Category "adarman" contains 31 posts

In [32]:
with open ("fa_stop_words.txt", "r") as fa_stop_words:
    stopwords = fa_stop_words.read().splitlines()

In [33]:
#for faster result, we define the following lines of code outside the 'preprocessing' function
normalizer = Normalizer()
lemmatizer = Lemmatizer()
stemmer = Stemmer()


def preprocessing(text):
    
    text = re.sub('<[^<]+?>','', text)
    text = ''.join(c for c in text if not c.isdigit())
    text = ''.join(c for c in text if c not in punctuation)
    text = normalizer.normalize(text)
    text = ' '.join(word for word in text.split() if word not in stopwords) # remove stopwors from text
    return text

## PreProcess

In [34]:
num_classes = len(unique_cats)

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each post.
MAX_SEQUENCE_LENGTH = 250
# The embedding dimension
EMBEDDING_DIM = 100

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(dataset["Cat"].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
# print(word_index)

X = tokenizer.texts_to_sequences(dataset["Cat"].values)
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Found 155 unique tokens.
Shape of data tensor: (165233, 250)


In [50]:
rows_of_train = 250000
rows_of_test = 75000

Y = pd.get_dummies(dataset['Cat']).values
print('Shape of label tensor:', Y.shape)

X_train = X[0:rows_of_train]
X_test = X[rows_of_train: rows_of_train + rows_of_test]

Y_train = Y[0:rows_of_train]
Y_test = Y[rows_of_train:rows_of_train + rows_of_test]

print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

Shape of label tensor: (165233, 113)
(165233, 250) (165233, 113)
(0, 250) (0, 113)


## Classification

In [57]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

epochs = 5
batch_size = 64
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

history = model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d_9 (Spatia  (None, 250, 100)         0         
 lDropout1D)                                                     
                                                                 
 lstm_10 (LSTM)              (None, 100)               80400     
                                                                 
 dense_9 (Dense)             (None, 6)                 606       
                                                                 
Total params: 5,081,006
Trainable params: 5,081,006
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


ValueError: in user code:

    File "/home/milad/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/milad/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/milad/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/milad/.local/lib/python3.10/site-packages/keras/engine/training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/milad/.local/lib/python3.10/site-packages/keras/engine/training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "/home/milad/.local/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, 